In [1]:
!pip install datasets
!pip install transformers
!pip install torch torchvision torchaudio

In [2]:
from datasets import load_dataset
#NOTE: they used train split, wiht ~3.5M examples
ds = load_dataset("wmt/wmt14", "de-en", split='train')
training_data = ds['translation']

In [3]:
import torch
import math

import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
torch.set_default_dtype(torch.float32)


num_layers = 6
seq_len = 512 # aka n_positions 
d_model = 512

class FFN(torch.nn.Module):
    def __init__(self):
        super().__init__()            
        
        self.d_ff = d_model*4
        self.W_1 = torch.nn.Linear(d_model, self.d_ff)
        self.relu = torch.nn.ReLU()
        self.W_2 = torch.nn.Linear(self.d_ff, d_model)
        
    def forward(self, x):
        # assert x.shape == (seq_len, d_model)    
        out = self.W_1(x)
        out = self.relu(out)
        out = self.W_2(out)

        return out

class MHA(torch.nn.Module):
    def __init__(self, h=8, has_mask=False):  
        super().__init__()          
        
        self.has_mask = has_mask
        self.d_k = d_model // h

        self.d_v = self.d_k        
        
        self.scale = 1/math.sqrt(self.d_k)

        assert d_model == 512
        assert self.d_k == 64
        assert self.d_v == self.d_k

        self.W_Q = torch.nn.Parameter(torch.nn.init.xavier_uniform_(torch.zeros(size=(d_model, self.d_k))))
        self.W_K = torch.nn.Parameter(torch.nn.init.xavier_uniform_(torch.zeros(size=(d_model, self.d_k))))
        self.W_V = torch.nn.Parameter(torch.nn.init.xavier_uniform_(torch.zeros(size=(d_model, self.d_v))))
        self.W_O = torch.nn.Parameter(torch.nn.init.xavier_uniform_(torch.zeros(size=(h*self.d_v, d_model))))
                
    #TODO: why separate x_k and x_v
    def attn(self, x_k, x_v):
        Q = x_k @ self.W_Q
        K = x_k @ self.W_K
        V = x_v @ self.W_V

        mask = torch.ones(seq_len, seq_len).to(device) 
        if self.has_mask:
            mask = torch.tril(mask)
        
        # softmax along dim=0 of Q@K.T => seq_len
        sm = torch.softmax(input=self.scale*mask*(Q@K.T), dim=0)
        head = sm @ V
        return head

    def forward(self, x_k, x_v):
        # assert x_k.shape == (seq_len, d_model)
        # assert x_v.shape == (seq_len, d_model)
        
        heads = torch.cat([self.attn(x_k, x_v) for _ in range(8)], dim=1)
        res = heads @ self.W_O
        return res

In [15]:
#vocab_dim = 37000
# vocab_dim = 50210
vocab_dim = 50257+1

class Embed(torch.nn.Module):
    def __init__(self, vocab_dim, d_model):
        super().__init__()
        self.emb = torch.nn.Embedding(num_embeddings=vocab_dim, embedding_dim=d_model, padding_idx=50257)

    def forward(self, x):
        # print(f"Embed: {self.emb}")
        # print(f"x: {x.shape}")
        res = self.emb(x)
        return res

class PosEmbed():
    def __init__(self):
        self.emb = torch.zeros(seq_len, d_model).to(device)

        for pos in range(seq_len):
            for i in range(d_model//2):
                arg = pos/(10000**(2*i/d_model))
                if i % 2 == 0:
                    self.emb[pos, 2*i] = math.sin(arg)
                else:
                    self.emb[pos, 2*i+1] = math.cos(arg)

    def fwd(self):
        # print(f"Pos embed type: {self.emb.shape}")
        # print(f"Position embedding: {self.emb[:3]}")        
        return self.emb

In [20]:
class Encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.M = MHA(has_mask=False)
        self.F = FFN()
        self.LN = torch.nn.LayerNorm(d_model)
        
    def forward(self, x):
        mha = self.M(x_k=x, x_v=x)

        sl1 = self.LN(x+mha)
        # print(sl1.shape)

        ffn = self.F(sl1)
        
        sl2 = self.LN(sl1 + ffn)
        # print(sl2.shape)

        return sl2

class Decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.M1 = MHA(has_mask=True)
        self.M2 = MHA(has_mask=False)
        self.F = FFN()
        self.LN = torch.nn.LayerNorm(d_model)        
    
    def forward(self, x, enc_out):
        mmha = self.M1(x_k=x, x_v=x)

        sl1 = self.LN(x + mmha)

        # print("sl1", sl1)

        mha = self.M2(x_k=enc_out, x_v=sl1)

        sl2 = self.LN(sl1 + mha)

        # print("sl2", sl2)        

        ffn = self.F(sl2)

        sl3 = self.LN(sl2 + ffn)

        return sl3

class EncoderDecoder(torch.nn.Module):      
    def __init__(self):
        super().__init__()        
        self.encs = torch.nn.ModuleList([Encoder() for _ in range(num_layers)])
        self.decs = torch.nn.ModuleList([Decoder() for _ in range(num_layers)])
        self.pos_embed = PosEmbed()
        

    def forward(self, x):
        # assert x.shape == (seq_len, d_model)

        enc_out = x
        for enc in self.encs:
            enc_out = enc(enc_out)

        # print("Enc_out", enc_out)
        dec_out = enc_out + self.pos_embed.fwd()

        
        for dec in self.decs:
            dec_out = dec(enc_out, dec_out)
            
        # print("Dec_out", dec_out)
        return dec_out   

In [24]:
class Transformer(torch.nn.Module):
    def __init__(self):
       super().__init__()
       self.input_embed = Embed(vocab_dim=vocab_dim, d_model=d_model)
       self.enc_dec = EncoderDecoder()       
       self.linear = torch.nn.Linear(d_model, vocab_dim) # (seq_len, d_model) -> (seq_len, vocab_dim)
       self.pos_embed = PosEmbed()        
        
    def forward(self, x):
       #Nit: torch.nn.Sequential
       # print(f"Inputs: {x.shape}")

       out = self.input_embed(x) + self.pos_embed.fwd()
       # print(f"Embedding: {out.shape}")                
       out = self.enc_dec(out) 
       # print(f"EncoderDecoder: {out.shape}")      
       out = self.linear(out)
       # print(f"Linear: {out.shape}")

       # Softmax over dim=0, seq_len
       softmax = torch.softmax(input=out, dim=0) 
       # print(f"Softmax: {softmax.shape}")        
       return softmax

#TODO: dropout 1.,2.,3.

In [25]:
training_data[:2]

[{'de': 'Wiederaufnahme der Sitzungsperiode',
  'en': 'Resumption of the session'},
 {'de': 'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.',
  'en': 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.'}]

In [26]:
from transformers import AutoTokenizer

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

model = Transformer().to(device)

tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
num_epochs = 100
criterion = torch.nn.CrossEntropyLoss()

optim = torch.optim.Adam(params=model.parameters(), betas=(0.9,0.98), eps=10E-9)
#TODO: learning rate
losses = []

for epoch in range(num_epochs+1):
    for sample in training_data[:2]:
        optim.zero_grad()

        tokenized_de = tokenizer.encode(sample['de'], padding='max_length', max_length=seq_len)
        tokenized_en = tokenizer.encode(sample['en'], padding='max_length', max_length=seq_len)
        
        targets = torch.tensor(tokenized_en).to(device)
        inputs = torch.tensor(tokenized_de).to(device)

        # print(f"inputs: {inputs.shape}")

        for input in inputs:
            if input > vocab_dim:
                print(input)

        for target in targets:
            if target > vocab_dim:
                print(target)
        
        # print(f"targets: {targets.shape}")
        # print(f"Tokenized targets: {len(targets)}")
        # print(targets)        

        outputs = model(inputs)

        # print(f"outputs: {outputs.shape}")        


        loss = criterion(outputs, targets)
        losses.append(loss)

        loss.backward()        
        optim.step()
        
    if epoch % 10 == 0:
        print(f"loss: {loss}")


loss: 10.824792861938477
loss: 10.824803352355957
loss: 10.824304580688477
loss: 10.824784278869629
loss: 10.823877334594727
loss: 10.824676513671875
loss: 10.825068473815918
loss: 10.822554588317871
loss: 10.824360847473145
loss: 10.82479190826416
loss: 10.822355270385742


In [9]:
import torch
print(torch.get_default_dtype())
print(torch.__version__)

torch.float32
2.4.1+cu121
